Compare a master image to a series of other images
--

In [20]:
import sys, os
from PIL import Image

import cv2
import matplotlib.pyplot as plt
import os
import datetime
from datetime import timedelta 
import pytz
import pandas as pd

In [2]:
def hammingDistance(s1, s2):
    """Return the Hamming distance between equal-length sequences"""
    if len(s1) != len(s2):
        raise ValueError("Undefined for sequences of unequal length")
    return sum(bool(ord(ch1) - ord(ch2)) for ch1, ch2 in zip(s1, s2))

In [3]:
def imfdt(imfile):
    #gets a datetime for an image file named like 20180101001002WIWR.jpg
    year = imfile[:4]
    month = imfile[4:6]
    day = imfile[6:8]
    hour = imfile[8:10]
    minute = imfile[10:12]
    second = imfile[12:14]
    dt = pytz.utc.localize(datetime.datetime(int(year), int(month), int(day), int(hour), int(minute), int(second)))
    return (dt)

In [4]:
def clahe(img):
    # CLAHE (Contrast Limited Adaptive Histogram Equalization)
    #https://stackoverflow.com/questions/19363293/whats-the-fastest-way-to-increase-color-image-contrast-with-opencv-in-python-c
    global img2
    clahe = cv2.createCLAHE(clipLimit=3., tileGridSize=(8,8))

    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)  # convert from BGR to LAB color space
    l, a, b = cv2.split(lab)  # split on 3 different channels

    l2 = clahe.apply(l)  # apply CLAHE to the L-channel

    lab = cv2.merge((l2,a,b))  # merge channels
    img2 = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)  # convert from LAB to BGR

In [7]:
def calchash(img):
    image = Image.open(img)

    image = image.resize((8, 8), Image.ANTIALIAS)
    image = image.convert("L")

    pixels = list(image.getdata())
    avg = sum(pixels) / len(pixels)

    bits = "".join(map(lambda pixel: '1' if (pixel < avg) else '0', pixels))  # '00010100...'
    hexhash = int(bits, 2).__format__('016x').upper()
    
    return (hexhash)

Get some images to work on
--

In [ ]:
#UTC date range for images to view
date1 = '2017-12-01'
date2 = '2017-12-31'
camera = 'WIWR'
hour1 = 0
hour2 = 2

#as mounted, this is image directory
camdir = '/run/user/1000/gvfs/ftp:host=ftp.geonet.org.nz/volcano_cams'

#directory to write crops and contrasted images
imgdir = '/home/sherburn/geonet/volcam_images'
crpdir = os.path.join(imgdir, 'crop')
cntdir = os.path.join(imgdir, 'contrast')

#read images, crop, write locally
for date in pd.date_range(date1, date2):
    year = date.year
    time_start = pytz.utc.localize(date + timedelta(hours=hour1))
    time_end = pytz.utc.localize(date + timedelta(hours=hour2))
    datestr = date.strftime('%Y%m%d')
    print (datestr)
    dydir = os.path.join(camdir, str(year), datestr, camera)
    
    for file in sorted(os.listdir(dydir)):
        if file.endswith('.jpg'):
            imfile = os.path.join(dydir, file)
            imtime = imfdt(file)
            if (imtime < time_end and imtime > time_start):
                img = cv2.imread(imfile)
                crop = img[400:700,750:1450]
                crfile = os.path.join(crpdir, file)
                cv2.imwrite(crfile, crop)
                
                clahe(crop) #contrast enhance cropped image
                ctfile = os.path.join(cntdir, file)
                cv2.imwrite(ctfile, img2)

Image comparison, with images without contrast enhancement
--

In [9]:
master = '/home/sherburn/geonet/volcam_images/crop/20171214002002WIWR.jpg'

In [26]:
#hash for master image
masthash = calchash(master)

allhash = []
files = []
#hash for all images (including master)
for file in sorted(os.listdir(crpdir)):
        if file.endswith('.jpg'):
            imfile = os.path.join(crpdir, file)
            hexhash = calchash(imfile)
            allhash.append(hexhash)
            files.append(imfile)

In [25]:
q1 = '/home/sherburn/geonet/volcam_images/crop/q1'
q2 = '/home/sherburn/geonet/volcam_images/crop/q2'
q3 = '/home/sherburn/geonet/volcam_images/crop/q3'
q4 = '/home/sherburn/geonet/volcam_images/crop/q4'

#calculate hamming distance for master compared to all
#print 'master:', imgmast, 'hash:', metric[0]
for idx, file in enumerate(files):
    hamdist = hammingDistance(masthash, allhash[idx])
    #print (idx, 'file:', file, 'hash:', allhash[idx], 'hamdist:', hamdist)
    
    #q4 images
    if (hamdist <= 3):
        print (idx, 'file:', file, 'hamdist:', hamdist, 'q4')
        cmd = 'cp '+file+' '+q4
        os.system(cmd)
        
    #q3 images
    if (hamdist >= 4 and hamdist <= 7):
        print (idx, 'file:', file, 'hamdist:', hamdist, 'q3')
        cmd = 'cp '+file+' '+q3
        os.system(cmd)
        
    #q2 images
    if (hamdist >= 8 and hamdist <= 11):
        print (idx, 'file:', file, 'hamdist:', hamdist, 'q2')
        cmd = 'cp '+file+' '+q2
        os.system(cmd)
        
    #q1 images
    if (hamdist >= 12):
        print (idx, 'file:', file, 'hamdist:', hamdist, 'q1')
        cmd = 'cp '+file+' '+q1
        os.system(cmd)

0 file: /home/sherburn/geonet/volcam_images/crop/20171201000002WIWR.jpg hamdist: 13 q1
1 file: /home/sherburn/geonet/volcam_images/crop/20171201001002WIWR.jpg hamdist: 14 q1
2 file: /home/sherburn/geonet/volcam_images/crop/20171201002002WIWR.jpg hamdist: 16 q1
3 file: /home/sherburn/geonet/volcam_images/crop/20171201003002WIWR.jpg hamdist: 10 q2
4 file: /home/sherburn/geonet/volcam_images/crop/20171201004002WIWR.jpg hamdist: 10 q2
5 file: /home/sherburn/geonet/volcam_images/crop/20171201005002WIWR.jpg hamdist: 15 q1
6 file: /home/sherburn/geonet/volcam_images/crop/20171201010002WIWR.jpg hamdist: 15 q1
7 file: /home/sherburn/geonet/volcam_images/crop/20171201011002WIWR.jpg hamdist: 15 q1
8 file: /home/sherburn/geonet/volcam_images/crop/20171201012002WIWR.jpg hamdist: 16 q1
9 file: /home/sherburn/geonet/volcam_images/crop/20171201013002WIWR.jpg hamdist: 14 q1
10 file: /home/sherburn/geonet/volcam_images/crop/20171201014002WIWR.jpg hamdist: 16 q1
11 file: /home/sherburn/geonet/volcam_imag

124 file: /home/sherburn/geonet/volcam_images/crop/20171211005002WIWR.jpg hamdist: 16 q1
125 file: /home/sherburn/geonet/volcam_images/crop/20171211010002WIWR.jpg hamdist: 16 q1
126 file: /home/sherburn/geonet/volcam_images/crop/20171211011002WIWR.jpg hamdist: 15 q1
127 file: /home/sherburn/geonet/volcam_images/crop/20171211012002WIWR.jpg hamdist: 11 q2
128 file: /home/sherburn/geonet/volcam_images/crop/20171211013002WIWR.jpg hamdist: 16 q1
129 file: /home/sherburn/geonet/volcam_images/crop/20171211014003WIWR.jpg hamdist: 12 q1
130 file: /home/sherburn/geonet/volcam_images/crop/20171211015003WIWR.jpg hamdist: 15 q1
131 file: /home/sherburn/geonet/volcam_images/crop/20171212000002WIWR.jpg hamdist: 13 q1
132 file: /home/sherburn/geonet/volcam_images/crop/20171212001002WIWR.jpg hamdist: 3 q4
133 file: /home/sherburn/geonet/volcam_images/crop/20171212002002WIWR.jpg hamdist: 7 q3
134 file: /home/sherburn/geonet/volcam_images/crop/20171212003002WIWR.jpg hamdist: 5 q3
135 file: /home/sherburn

246 file: /home/sherburn/geonet/volcam_images/crop/20171221011002WIWR.jpg hamdist: 12 q1
247 file: /home/sherburn/geonet/volcam_images/crop/20171221012002WIWR.jpg hamdist: 7 q3
248 file: /home/sherburn/geonet/volcam_images/crop/20171221013002WIWR.jpg hamdist: 7 q3
249 file: /home/sherburn/geonet/volcam_images/crop/20171221014002WIWR.jpg hamdist: 8 q2
250 file: /home/sherburn/geonet/volcam_images/crop/20171221015002WIWR.jpg hamdist: 11 q2
251 file: /home/sherburn/geonet/volcam_images/crop/20171222000002WIWR.jpg hamdist: 16 q1
252 file: /home/sherburn/geonet/volcam_images/crop/20171222001002WIWR.jpg hamdist: 15 q1
253 file: /home/sherburn/geonet/volcam_images/crop/20171222002002WIWR.jpg hamdist: 15 q1
254 file: /home/sherburn/geonet/volcam_images/crop/20171222003002WIWR.jpg hamdist: 15 q1
255 file: /home/sherburn/geonet/volcam_images/crop/20171222004002WIWR.jpg hamdist: 15 q1
256 file: /home/sherburn/geonet/volcam_images/crop/20171222005002WIWR.jpg hamdist: 15 q1
257 file: /home/sherburn

369 file: /home/sherburn/geonet/volcam_images/crop/20171231014003WIWR.jpg hamdist: 9 q2
370 file: /home/sherburn/geonet/volcam_images/crop/20171231015003WIWR.jpg hamdist: 7 q3


Image comparison, with images with contrast enhancement
--

In [27]:
master = '/home/sherburn/geonet/volcam_images/contrast/20171214002002WIWR.jpg'

In [28]:
#hash for master image
masthash = calchash(master)

allhash = []
files = []
#hash for all images (including master)
for file in sorted(os.listdir(cntdir)):
        if file.endswith('.jpg'):
            imfile = os.path.join(cntdir, file)
            hexhash = calchash(imfile)
            allhash.append(hexhash)
            files.append(imfile)

In [29]:
q1 = '/home/sherburn/geonet/volcam_images/contrast/q1'
q2 = '/home/sherburn/geonet/volcam_images/contrast/q2'
q3 = '/home/sherburn/geonet/volcam_images/contrast/q3'
q4 = '/home/sherburn/geonet/volcam_images/contrast/q4'

#calculate hamming distance for master compared to all
#print 'master:', imgmast, 'hash:', metric[0]
for idx, file in enumerate(files):
    hamdist = hammingDistance(masthash, allhash[idx])
    #print (idx, 'file:', file, 'hash:', allhash[idx], 'hamdist:', hamdist)
    
    #q4 images
    if (hamdist <= 3):
        print (idx, 'file:', file, 'hamdist:', hamdist, 'q4')
        cmd = 'cp '+file+' '+q4
        os.system(cmd)
        
    #q3 images
    if (hamdist >= 4 and hamdist <= 7):
        print (idx, 'file:', file, 'hamdist:', hamdist, 'q3')
        cmd = 'cp '+file+' '+q3
        os.system(cmd)
        
    #q2 images
    if (hamdist >= 8 and hamdist <= 11):
        print (idx, 'file:', file, 'hamdist:', hamdist, 'q2')
        cmd = 'cp '+file+' '+q2
        os.system(cmd)
        
    #q1 images
    if (hamdist >= 12):
        print (idx, 'file:', file, 'hamdist:', hamdist, 'q1')
        cmd = 'cp '+file+' '+q1
        os.system(cmd)

0 file: /home/sherburn/geonet/volcam_images/contrast/20171201000002WIWR.jpg hamdist: 14 q1
1 file: /home/sherburn/geonet/volcam_images/contrast/20171201001002WIWR.jpg hamdist: 11 q2
2 file: /home/sherburn/geonet/volcam_images/contrast/20171201002002WIWR.jpg hamdist: 15 q1
3 file: /home/sherburn/geonet/volcam_images/contrast/20171201003002WIWR.jpg hamdist: 10 q2
4 file: /home/sherburn/geonet/volcam_images/contrast/20171201004002WIWR.jpg hamdist: 12 q1
5 file: /home/sherburn/geonet/volcam_images/contrast/20171201005002WIWR.jpg hamdist: 14 q1
6 file: /home/sherburn/geonet/volcam_images/contrast/20171201010002WIWR.jpg hamdist: 15 q1
7 file: /home/sherburn/geonet/volcam_images/contrast/20171201011002WIWR.jpg hamdist: 15 q1
8 file: /home/sherburn/geonet/volcam_images/contrast/20171201012002WIWR.jpg hamdist: 16 q1
9 file: /home/sherburn/geonet/volcam_images/contrast/20171201013002WIWR.jpg hamdist: 14 q1
10 file: /home/sherburn/geonet/volcam_images/contrast/20171201014002WIWR.jpg hamdist: 16 q

122 file: /home/sherburn/geonet/volcam_images/contrast/20171211003002WIWR.jpg hamdist: 16 q1
123 file: /home/sherburn/geonet/volcam_images/contrast/20171211004002WIWR.jpg hamdist: 16 q1
124 file: /home/sherburn/geonet/volcam_images/contrast/20171211005002WIWR.jpg hamdist: 16 q1
125 file: /home/sherburn/geonet/volcam_images/contrast/20171211010002WIWR.jpg hamdist: 15 q1
126 file: /home/sherburn/geonet/volcam_images/contrast/20171211011002WIWR.jpg hamdist: 14 q1
127 file: /home/sherburn/geonet/volcam_images/contrast/20171211012002WIWR.jpg hamdist: 14 q1
128 file: /home/sherburn/geonet/volcam_images/contrast/20171211013002WIWR.jpg hamdist: 14 q1
129 file: /home/sherburn/geonet/volcam_images/contrast/20171211014003WIWR.jpg hamdist: 16 q1
130 file: /home/sherburn/geonet/volcam_images/contrast/20171211015003WIWR.jpg hamdist: 14 q1
131 file: /home/sherburn/geonet/volcam_images/contrast/20171212000002WIWR.jpg hamdist: 16 q1
132 file: /home/sherburn/geonet/volcam_images/contrast/20171212001002W

242 file: /home/sherburn/geonet/volcam_images/contrast/20171221003003WIWR.jpg hamdist: 11 q2
243 file: /home/sherburn/geonet/volcam_images/contrast/20171221004003WIWR.jpg hamdist: 9 q2
244 file: /home/sherburn/geonet/volcam_images/contrast/20171221005003WIWR.jpg hamdist: 11 q2
245 file: /home/sherburn/geonet/volcam_images/contrast/20171221010003WIWR.jpg hamdist: 6 q3
246 file: /home/sherburn/geonet/volcam_images/contrast/20171221011002WIWR.jpg hamdist: 12 q1
247 file: /home/sherburn/geonet/volcam_images/contrast/20171221012002WIWR.jpg hamdist: 12 q1
248 file: /home/sherburn/geonet/volcam_images/contrast/20171221013002WIWR.jpg hamdist: 13 q1
249 file: /home/sherburn/geonet/volcam_images/contrast/20171221014002WIWR.jpg hamdist: 7 q3
250 file: /home/sherburn/geonet/volcam_images/contrast/20171221015002WIWR.jpg hamdist: 7 q3
251 file: /home/sherburn/geonet/volcam_images/contrast/20171222000002WIWR.jpg hamdist: 16 q1
252 file: /home/sherburn/geonet/volcam_images/contrast/20171222001002WIWR.

369 file: /home/sherburn/geonet/volcam_images/contrast/20171231014003WIWR.jpg hamdist: 13 q1
370 file: /home/sherburn/geonet/volcam_images/contrast/20171231015003WIWR.jpg hamdist: 10 q2
